<h2>Create Cluster in Python</h2>

In [1]:
from dask_gateway import Gateway

In [2]:
from dask_gateway import GatewayCluster

In [3]:
gateway = Gateway()

In [4]:
options = gateway.cluster_options()

In [5]:
options

In [6]:
cluster = gateway.new_cluster()

In [7]:
cluster

In [8]:
gateway.list_clusters()

[ClusterReport<name=daskhub.53c1ad490ac342138ecefad4abbf6758, status=RUNNING>]

In [9]:
# Dask helpers
def slam(client,action,data,partition_factor=1.5,dbg=0):
    np = sum(client.nthreads().values())
    if dbg>0: # a
        print('slam: np = %i'%np)
    shard_bounds = [int(i*len(data)/(1.0*partition_factor*np)) for i in range(int(partition_factor*np))] 
    if shard_bounds[-1] != len(data):
        if dbg>0: # b
            print('a sb[-1]: ',shard_bounds[-1],len(data))
        shard_bounds = shard_bounds + [len(data)]
        if dbg>0: # c
            print('sb: ',shard_bounds)
    data_shards = [data[shard_bounds[i]:shard_bounds[i+1]] for i in range(len(shard_bounds)-1)]
    if dbg>0: # d
        print('ds len:        ',len(data_shards))
        print('ds item len:   ',len(data_shards[0]))
        print('ds type:       ',type(data_shards[0]))
        try:
            print('ds dtype:      ',data_shards[0].dtype)
        except:
            pass
    big_future = client.scatter(data_shards)
    results    = client.map(action,big_future)
    return results


In [10]:
import boto3
import starepandas
import sqlalchemy
import os
import tempfile
import pandas
import pystare as ps

def action(data_shard):
    ret = []
    
    s3_client = boto3.client('s3')
    bucket_name='daskhub-data'
    key='s3://daskhub-data/MODAPS/CATALOGS/MOD05_L2/2019/356/catalogue.sqlite'
    tmpdir = tempfile.TemporaryDirectory()
    filename='catalogue.sqlite'
    fullFilename = os.path.join(tmpdir.name,filename)
    s3_client.download_file(bucket_name,key,fullFilename)   
    
    if False:
    
        db_path = fullFilename
        uri = 'sqlite:///{db_path}'.format(db_path=db_path)
        engine = sqlalchemy.create_engine(uri)
        catalog2019356 = pandas.read_sql(sql='catalogue', con=engine)
    
        m2_ds = starepandas.nc4_Dataset_wrapper('s3://daskhub-data/GESDISC/MERRA2/2019/MERRA2_400.tavg1_2d_slv_Nx.20191222.nc4')
    
        for trixel in data_shard:
            trixel_intersects = catalogue2019356.stare_intersects(trixel)
            granule_subset2019356    = catalogue2019356[trixel_intersects].granule_name
            granule_names2019356     = list(granule_subset2019356)
        
            data_objs2019356 = [modis_data(name) for name in granule_names2019356]
            intersections2019356 = [
                ps.intersect(trixel,catalog2019356['stare_cover'][i].data,1) for i in granule_subset.keys()
            ]
            intersected_objs2019356 = [modis_subset(data_objs2019356[i],intersections2019356[i]) for i in range(len(intersections2019356))]
        
            m2_data_size = sum([o.subset_wvir.size for o in intersected_objs]) # number of points found
        
            ret.append([trixel,m2_data_size])
        
        m2_ds.close()
        
    tmpdir.cleanup()
        
    return ret

    

In [11]:
client = cluster.get_client()

/home/jovyan/users_conda_envs/stare-1/lib/python3.7/site-packages/distributed/client.py:1136: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| numpy   | 1.19.2 | 1.19.1    | 1.19.1  |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [12]:
if False:
    m2_sc=merra2_sidecar(cover_resolution=3)
    data = m2_sc.cover
    
if True:
    data = list(range(10))
    
data

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [13]:
results = slam(client,action,data,partition_factor=1.5,dbg=1)

slam: np = 2
a sb[-1]:  6 10
sb:  [0, 3, 6, 10]
ds len:         3
ds item len:    3
ds type:        <class 'list'>


In [14]:
client.gather(results)

PermissionError: [Errno 13] Permission denied: '/var/run/secrets/eks.amazonaws.com/serviceaccount/token'

In [15]:
client.close()

In [38]:
gateway.list_clusters()

[ClusterReport<name=daskhub.cd0349c80f7e4320a26a39c1857c015b, status=RUNNING>]

In [16]:
cluster.close()

In [ ]:
https://daskhub-data.s3.amazonaws.com/significant_month.csv